In [6]:
import json

with open("xquad_en_train.json") as f:
    examples = json.load(f)["examples"]
contexts = [example["context"] for example in examples]
len(contexts), contexts[0]

(103,
 'On 20 May 2009, May and Taylor performed "We Are the Champions" live on the season finale of American Idol with winner Kris Allen and runner-up Adam Lambert providing a vocal duet. In mid-2009, after the split of Queen + Paul Rodgers, the Queen online website announced a new greatest hits compilation named Absolute Greatest. The album was released on 16 November and peaked at number 3 in the official UK Chart. The album contains 20 of Queen\'s biggest hits spanning their entire career and was released in four different formats: single disc, double disc (with commentary), double disc with feature book, and a vinyl record. Prior to its release, a competition was run by Queen online to guess the track listing as a promotion for the album.')

In [15]:
with open("en_contexts.txt", "w") as f:
    for context in contexts: f.write(context+"\n")

In [24]:
def bleu_scores(language):
    marian_train_path = f"../{language}/xquad_en2{language}_train.json"
    nllb_train_path = f"../{language}/xquad_en2{language}_nllb_train.json"
    gtrans_context_path = f"{language}_contexts.txt"
    with open(marian_train_path) as f: 
        marian_contexts = [example["context"] for example in json.load(f)["examples"]]
    with open(nllb_train_path) as f: 
        nllb_contexts = [example["context"] for example in json.load(f)["examples"]]
    with open(gtrans_context_path) as f:
        gtrans_contexts = f.readlines()
    # find position of unique contexts in gtrans_contexts
    unique_contexts_idx = [gtrans_contexts.index(context) for context in list(set(gtrans_contexts))]
    marian_contexts = [marian_contexts[i] for i in unique_contexts_idx]
    nllb_contexts = [nllb_contexts[i] for i in unique_contexts_idx]
    gtrans_contexts = [gtrans_contexts[i] for i in unique_contexts_idx]
    len(marian_contexts), len(nllb_contexts), len(gtrans_contexts)

    assert(len(marian_contexts) == len(nllb_contexts) == len(gtrans_contexts))
    from sacrebleu.metrics import BLEU
    sacrebleu_bleu = BLEU(effective_order=True)
    marian_bleu = []
    nllb_bleu = []
    for i in range(len(marian_contexts)):
        marian_bleu.append(sacrebleu_bleu.sentence_score(marian_contexts[i], [gtrans_contexts[i]]).score)
        nllb_bleu.append(sacrebleu_bleu.sentence_score(nllb_contexts[i], [gtrans_contexts[i]]).score)
    return {
        "MarianMT": sum(marian_bleu)/len(marian_bleu),
        "NLLB": sum(nllb_bleu)/len(nllb_bleu)
    }

In [27]:
scores = {}
for language in ["de", "et", "fi", "fr", "id", "sv", "xh"]:
    scores[language] = bleu_scores(language)
import pandas as pd
df = pd.DataFrame(scores).T
df

,MarianMT,NLLB
de,40.571668,34.138689
et,28.279430,24.338929
fi,23.963931,22.802352
fr,44.993039,37.994392
id,19.998780,46.976076
sv,56.655614,41.352073
xh,8.422087,23.886481
